In [1]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import SearchRequest, NamedVectorStruct
import numpy as np

c:\Users\Zaloon098\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sentences = [
    "What a shitty restaurant! The food was horrible. I hope a health inspection catches you by surprise!",
    "This restaurant has impressive views i really enjoyed coming here for a lunch",
]

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = model.encode(sentences)
print(len(embeddings[0]))

384


In [3]:
qdrant = QdrantClient("localhost", port=6333)

In [4]:
embeddings[0].tolist()[:3]

[0.012072589248418808, 0.089756079018116, 0.04744037240743637]

In [5]:
SearchRequest(vector=[1.0, 2.0, 3.0], limit=1)

SearchRequest(vector=[1.0, 2.0, 3.0], filter=None, params=None, limit=1, offset=0, with_payload=None, with_vector=None, score_threshold=None)

In [6]:
requests = [
    SearchRequest(vector=v.tolist(), limit=1, with_payload=True) for v in embeddings
]

In [7]:
hits_unflattened = qdrant.search_batch(
    collection_name="restaurant_review_answers", requests=requests
)

In [8]:
from itertools import chain

hits = list(chain.from_iterable(hits_unflattened))
hits

[ScoredPoint(id=21, version=0, score=0.5444159, payload={'answer': "We're sorry to hear that your visit to our beach restaurant didn't meet your expectations. We appreciate your feedback and will work hard to address the issues you mentioned. We hope to have another opportunity to serve you better in the future.\n"}, vector=None),
 ScoredPoint(id=18, version=0, score=0.6271736, payload={'answer': "Thank you for your honest review of our beach restaurant. We're committed to delivering a memorable dining experience, and we'll use your feedback to identify areas for improvement.\n"}, vector=None)]

In [9]:
for sentence, hit in zip(sentences, hits):
    print("Review: " + sentence)
    print(f"Answer (score: {hit.score}): " + hit.payload["answer"])

Review: What a shitty restaurant! The food was horrible. I hope a health inspection catches you by surprise!
Answer (score: 0.5444159): We're sorry to hear that your visit to our beach restaurant didn't meet your expectations. We appreciate your feedback and will work hard to address the issues you mentioned. We hope to have another opportunity to serve you better in the future.

Review: This restaurant has impressive views i really enjoyed coming here for a lunch
Answer (score: 0.6271736): Thank you for your honest review of our beach restaurant. We're committed to delivering a memorable dining experience, and we'll use your feedback to identify areas for improvement.

